In [34]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv


In [35]:
load_dotenv()  # .env 파일의 환경변수 로드
api_key = os.getenv("api_key")

In [36]:
# ========== 1. 경로 설정 ==========
PDF_PATH = r"C:\Project\web\round12\KB 실버암 간편건강보험Plus.pdf"


In [37]:
# ========== 2. PDF 로드 ==========
loader = PyPDFLoader(PDF_PATH)
pages = loader.load_and_split()

In [38]:
# ========== 3. 텍스트 청크 분할 ==========
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
docs = text_splitter.split_documents(pages)


In [39]:
# ========== 4. OpenAI 임베딩 ==========
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key="api_key" # 환경 변수에서 불러오기
)

In [40]:
# ========== 5. 벡터 저장소 생성 (FAISS) ==========
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever()


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: api_key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# 6. Claude 3.7 Sonnet 모델 설정
llm = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",  # 최신 Claude 3.7 Sonnet 모델
    temperature=0,
    max_tokens=1024,
    anthropic_api_key="api-key"
)

In [ ]:
# ========== 7. 프롬프트 템플릿 ==========
def rag_prompt(context, question):
    return f"""당신은 보험 약관 분석 전문가입니다.
아래 컨텍스트를 참고해 질문에 답하세요.
문서에 없는 내용은 '문서에서 관련 정보를 찾을 수 없습니다'라고 답변하세요.

컨텍스트:
{context}

질문:
{question}

답변:"""

In [ ]:
# ========== 8. RAG 체인 ==========
def rag_chain(question):
    # 1) 관련 문서 검색
    relevant_docs = retriever.get_relevant_documents(question)
    context_text = "\n\n".join([doc.page_content for doc in relevant_docs])

    # 2) Claude 프롬프트 생성
    prompt_text = rag_prompt(context_text, question)

    # 3) Claude 호출
    response = llm.invoke(prompt_text)
    return response.content

In [ ]:
# ========== 9. 실행 ==========
if __name__ == "__main__":
    # 예시 질문
    question = "평형기능의 장해가 있는거 같아 혹시 기준표나 점수가 있을까?"
    answer = rag_chain(question)
    print(answer)

C:\Users\KDP014\AppData\Local\Temp\ipykernel_32588\2223770528.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)


네, 평형기능의 장해에 대한 기준표와 점수가 문서에 있습니다.

문서에 따르면, '평형기능에 장해를 남긴 때'라 함은 전정기관 이상으로 보행 등 일상생활이 어려운 상태로, 평형장해 평가항목별 합산점수가 30점 이상인 경우를 말합니다.

평가항목별 점수는 다음과 같습니다:
- 검사소견: 양측 전정기능 소실(14점), 양측 전정기능 감소(10점), 일측 전정기능(점수 미기재)

문서에 제공된 정보는 일부만 있어 전체 평가항목과 점수를 모두 확인할 수는 없습니다.


In [ ]:
# ========== 9. 실행 ==========
if __name__ == "__main__":
    # 예시 질문
    question = "암보험 가입 후 2개월이 지나서 위암 판정을 받았어 보상받을 수 있나요?"
    answer = rag_chain(question)
    print(answer)

문서에 따르면, 암보험 가입 후 90일간은 보장이 제외됩니다. 민원사례에서도 "A씨는 암보험 가입 후 2개월이 지나서 위암을 판정받아 보험회사에 암 진단비를 청구⇒ 보험회사는 보험가입 후 90일이 경과하지 않아 보험금 지급이" 라고 언급되어 있습니다.

따라서 암보험 가입 후 2개월(90일 미만)이 지난 시점에 위암 판정을 받은 경우에는 보상을 받을 수 없습니다. 보장을 받기 위해서는 보험 가입 후 90일이 경과해야 합니다.


In [ ]:
# ========== 9. 실행 ==========
if __name__ == "__main__":
    # 예시 질문
    question = """
1. 암보험 가입 후 2개월이 지나서 위암 판정을 받았어 보상받을 수 있나요?
2. 평형기능의 장해가 있는거 같아 혹시 기준표나 점수가 있을까?
3. 제자리암 진단비 지급금액에 대해서 자세히 알려줘
"""
    answer = rag_chain(question)
    print(answer)

질문에 대한 답변을 드리겠습니다.

1. 암보험 가입 후 2개월이 지나서 위암 판정을 받았어 보상받을 수 있나요?
제공된 문서에 따르면 암에 대한 보장개시일(암보장개시일)이 있으며, 가입 후 90일간 보장이 제외된다고 명시되어 있습니다. 따라서 가입 후 2개월(약 60일)이 지난 시점에서 위암 판정을 받으셨다면, 아직 보장개시일(90일) 이전이므로 보상을 받을 수 없습니다.

2. 평형기능의 장해가 있는거 같아 혹시 기준표나 점수가 있을까?
문서에서 관련 정보를 찾을 수 없습니다.

3. 제자리암 진단비 지급금액에 대해서 자세히 알려줘
문서에서 관련 정보를 찾을 수 없습니다. 제공된 문서에는 "3대특정암", "남성생식기관련(전립선,음경,고환)암", "암(기타피부암 및 갑상선암 제외)"에 대한 보장 내용은 있으나, 제자리암 진단비에 대한 구체적인 정보는 포함되어 있지 않습니다.
